## Import and Initializations

In [1]:
import pandas as pd
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
import param 
import holoviews as hv

## Functions

In [2]:
def add_status_column(df, value):
    df['status'] = value
    return df

In [3]:
def add_period_id(col, dictionary):
    return df_combine[col].apply(lambda x: dictionary[x])

In [4]:
def rename_combine_idf(df_casual, df_registered):
    
    # Rename columns 'casual' and 'registered' for 'value'
    df_casual.rename(columns = {'casual' : 'value'}, inplace = True)
    df_registered.rename(columns = {'registered' : 'value'}, inplace = True)

    # Combine casual and registered
    return pd.concat([df_casual, df_registered], axis = 0, ignore_index = False)

## Data Management

In [5]:
df_bike_hour = pd.DataFrame(pd.read_csv('dataset/bike_sharing_hour_transformed.csv'))

## Plot Preparation

### Labellings

In [6]:
# Labels for category order when creating plots
# Period related category
years = ['2011', '2012']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
seasons = ['Spring', 'Summer', 'Autumn', 'Winter']
weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday', 'Saturday']
hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
rentals = ['casual', 'registered']

# Situation related category
weathersits = ['Light', 'Medium', 'Heavy', 'Very Heavy']                       
holidays = ['No', 'Yes']
working_days = ['No', 'Yes']

### Dictionaries

In [7]:
# Labelling for plots
plot_labels = {'yr': 'Year', 'mnth' : 'Month', 'hr' : "Hour", 'weekday' : 'Weekday', 
               'season' : 'Seasons', 'weathesit' : 'Weather Situation', 'cnt' : 'Total Rentals', 'casual' : 'Total Casual Rentals', 'registered' : 'Total Registered Rentals'}

month_dict = {'January' : 1, 'February' : 2, 'March' : 3, 'April' : 4, 'May' : 5, 'June' : 6, 
              'July' : 7, 'August' : 8, 'September' : 9 , 'October' : 10, 'November' : 11, 'December' : 12}

weekday_dict = {'Sunday' : 1, 'Monday' : 2, 'Tuesday' : 3, 'Wednesday' : 4, 'Thrusday' : 5, 'Friday' : 6, 'Saturday' : 7}

### Column conversion

In [8]:
# Set Year to String for better plot axis
df_bike_hour['yr'] = df_bike_hour['yr'].astype('str')

## Interactive plots

### Widgets

In [9]:
select_year = pn.widgets.MultiSelect(name = 'Year', value = years, options = years)
select_month = pn.widgets.MultiSelect(name = 'Month', value = months, options = months)
select_weekday = pn.widgets.MultiSelect(name = 'Weekday', value = weekdays, options = weekdays)
select_rental = pn.widgets.MultiSelect(name = 'Rental', value = rentals, options = rentals)

### Interactive Heatmap of Hour vs Month for Total Rental [Casual, Registered] 

In [10]:
# Groupby 'casual' and 'registered' dataframes
df_heatmap_casual = df_bike_hour.groupby(['yr', 'mnth', 'hr'])['casual'].sum().reset_index()
df_heatmap_registered = df_bike_hour.groupby(['yr', 'mnth', 'hr'])['registered'].sum().reset_index()

# Split the dataframe into 2 dataframe ['casual' and 'registered']
df_heatmap_casual = add_status_column(df_heatmap_casual, 'casual')
df_heatmap_registered = add_status_column(df_heatmap_registered, 'registered')

# Relabel columns on both dataframes, combine dataframes and get interactive dataframe
df_combine = rename_combine_idf(df_heatmap_casual, df_heatmap_registered)

# Add period IDs
df_combine['month_id'] = add_period_id('mnth', month_dict)

# Relabel columns on both dataframes, combine dataframes and get interactive dataframe
idf_bike_heatmap = df_combine.reset_index().interactive()

# Create bike pipeline 
bike_pipeline = (idf_bike_heatmap[(idf_bike_heatmap.yr.isin(select_year) & idf_bike_heatmap.mnth.isin(select_month) & idf_bike_heatmap.status.isin(select_rental))].sort_values(by = 'month_id'))
heatmap_hr_mnth = bike_pipeline.hvplot(title = 'Hour vs Month for Total Rental',
                                       kind = 'scatter', x = 'hr', y = 'mnth', color = 'value', cmap = 'Inferno', 
                                       xlabel = 'Hour', ylabel = 'Month', clabel = 'Total Rental')

### Interactive Heatmap of Hour vs Weekday for Total Rental [Casual, Registered] 

In [11]:
# Groupby Weekday and Hour for Casual and Registered Users
df_heatmap_casual = df_bike_hour.groupby(['yr', 'weekday', 'hr'])['casual'].sum().reset_index()
df_heatmap_registered = df_bike_hour.groupby(['yr', 'weekday', 'hr'])['registered'].sum().reset_index()

# Split the dataframe into 2 dataframe ['casual' and 'registered']
df_heatmap_casual = add_status_column(df_heatmap_casual, 'casual')
df_heatmap_registered = add_status_column(df_heatmap_registered, 'registered')

# Relabel columns on both dataframes, combine dataframes and get interactive dataframe
df_combine = rename_combine_idf(df_heatmap_casual, df_heatmap_registered)

# Add period IDs
df_combine['weekday_id'] = add_period_id('weekday', weekday_dict)

# Relabel columns on both dataframes, combine dataframes and get interactive dataframe
idf_bike_heatmap = df_combine.reset_index().interactive()

# Create bike pipeline
bike_pipeline = (idf_bike_heatmap[(idf_bike_heatmap.yr.isin(select_year) & idf_bike_heatmap.weekday.isin(select_weekday) & idf_bike_heatmap.status.isin(select_rental))].sort_values(by = 'weekday_id'))
heatmap_hr_weekday = bike_pipeline.hvplot(title = 'Hour vs Weekday for Total Rental',
                                          kind = 'scatter', x = 'hr', y = 'weekday', color = 'value', cmap = 'Inferno', 
                                          xlabel = 'Hour', ylabel = 'Weekday', clabel = 'Total Rental')

### Interactive Heatmap of Weekday vs Month for Total Rental [Casual, Registered] 

In [12]:
df_heatmap_casual = df_bike_hour.groupby(['yr', 'mnth', 'weekday'])['casual'].sum().reset_index()
df_heatmap_registered = df_bike_hour.groupby(['yr', 'mnth', 'weekday'])['registered'].sum().reset_index()

# Split the dataframe into 2 dataframe ['casual' and 'registered']
df_heatmap_casual = add_status_column(df_heatmap_casual, 'casual')
df_heatmap_registered = add_status_column(df_heatmap_registered, 'registered')

# Relabel columns on both dataframes, combine dataframes and get interactive dataframe
df_combine = rename_combine_idf(df_heatmap_casual, df_heatmap_registered)

# Add period IDs
df_combine['weekday_id'] = add_period_id('weekday', weekday_dict)
df_combine['month_id'] = add_period_id('mnth', month_dict)

# Interactive heatmap
idf_bike_heatmap = df_combine.reset_index().interactive()

bike_pipeline = (idf_bike_heatmap[(idf_bike_heatmap.yr.isin(select_year) & idf_bike_heatmap.mnth.isin(select_month) & idf_bike_heatmap.weekday.isin(select_weekday) & idf_bike_heatmap.status.isin(select_rental))].sort_values(by = ['weekday_id', 'month_id']))
heatmap_weekday_mnth = bike_pipeline.hvplot(title = 'Weekday vs Month for Total Rental', 
                                            kind = 'scatter', x = 'weekday', y = 'mnth', color = 'value', cmap = 'Inferno', 
                                            xlabel = 'Weekday', ylabel = 'Month', clabel = 'Total Rental')

## Dashboard

In [13]:
# run on terminal with command > panel serve bike_sharing_interactive.ipynb
template = pn.template.FastListTemplate(title = 'Bike Sharing Data Exploration',  
                                        sidebar = [pn.Column(select_year, select_month, select_weekday, select_rental)],
                                        main = [pn.Row(heatmap_hr_mnth.panel(width = 650, height = 500, margin = (0,25))), 
                                                pn.Row(heatmap_hr_weekday.panel(width = 650, height = 500, margin = (0,25))),
                                                pn.Row(heatmap_weekday_mnth.panel(width = 650, height = 500, margin = (0,25)))], 
                                        accent_base_color = '88d8b0', header_background = '88d8b0')
template.servable()

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [140276936869824] Row
        [0] ParamFunction(function, height=500, margin=(0, 25), sizing_mode='fixed', width=650)
    [140276965039744] Row
        [0] ParamFunction(function, height=500, margin=(0, 25), sizing_mode='fixed', width=650)
    [140277052939328] Row
        [0] ParamFunction(function, height=500, margin=(0, 25), sizing_mode='fixed', width=650)
    [140276936869920] Column
        [0] MultiSelect(name='Year', options=['2011', '2012'], value=['2011', '2012'])
        [1] MultiSelect(name='Month', options=['January', 'February', ...], value=['January', 'February', ...])
        [2] MultiSelect(name='Weekday', options=['Sunday', 'Monday', ...], value=['Sunday', 'Monday', ...])
        [3] MultiSelect(name='Rental', options=['casual', 'registered'], value=['casual', 'registered'])